In [ ]:
# %pip install dotenv langchain langchain-google
# %pip install langchain_community 
# %pip install tavily-python
# %pip install langchain-community beautifulsoup4
# %pip install langchain_google_genai
# %conda install weasyprint  # need to run this as well
# %pip install markdown weasyprint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dotenv]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.14.2-py3-none-any.whl (106 kB)
Using cached soupsieve-2.8-py3-none-any.whl (36 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [beautifulsoup4]m [beautifulsoup4]
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 52.4 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [langchain_google_genai]e-ai-generativelanguage]
Note: you may need to restart the kernel to use updated packages.
Solving environment: don

# Resume writer

This notebook helps you create a tailored resume and cover letter for a specific job description using AI.
It uses LangChain and Google GenAI to process your existing resume and the job description, extracting relevant information and formatting it appropriately.
It expects a version of the resume in pdf or docx format.
The resume should contain sections like "Work Experience", "Skills", "Education", and your personal information, etc.
It also expects a job description as a URL (tested it with LinkedIn but should work with others too).

It performs web search to get information about the hiring company to better tailor the resume and cover letter.

What you need to provide:
- Your resume in pdf or docx format
- A job description URL 
- Add a `.env` file with your Google GenAI API key and the Tavily API key as follows
- Modify the `Setup variables` cell to point to your resume and the job description URL


In [5]:

from dotenv import load_dotenv
import os
import sys
import json
from langchain_community.tools import TavilySearchResults
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent


## Setup variables

Setup the keys we need (for Tavily and GOOGLE gemini)

We need to store the keys to a file named `.env` in the same directory as this notebook with the following content:

```
TAVILY_API_KEY=your_tavily_api_key
GOOGLE_API_KEY=your_google_api_key
```


In [3]:
load_dotenv()
google_key=os.getenv("GOOGLE_API_KEY")
tavily_key=os.getenv("TAVILY_API_KEY")

In [6]:
# setup the variables
# this will become some sort of an input

company=None
resume="resume.pdf"
jd=None
title=None
job_level=None # e.g director etc
jd_url="https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4314922582"
# jd_url="https://www.linkedin.com/jobs/view/4314922582/"

In [7]:
# create the model to use
model=init_chat_model("google_genai:gemini-2.5-flash-lite")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [76]:
# useful functions
def strip_markdown_backticks(text):
    """
    Strip markdown code block backticks from text.
    Handles both triple backticks (```) and triple backticks with language specifier.
    """
    text = text.strip()
    
    # Remove starting backticks (with or without language specifier)
    if text.startswith('```'):
        # Find the end of the first line (after language specifier if present)
        first_newline = text.find('\n')
        if first_newline != -1:
            text = text[first_newline + 1:]
        else:
            # No newline found, just remove the backticks
            text = text[3:]
    
    # Remove ending backticks
    if text.endswith('```'):
        text = text[:-3]
    
    return text.strip()


def markdown_to_pdf(markdown_text, output_path):
    """
    Convert markdown to PDF using weasyprint (best formatting)
    """
    import markdown
    from weasyprint import HTML, CSS
    from weasyprint.text.fonts import FontConfiguration
    
    # Convert markdown to HTML
    html = markdown.markdown(markdown_text)
    
    # Add basic styling
    styled_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <style>
            body {{ 
                font-family: Arial, sans-serif; 
                margin: 40px; 
                line-height: 1; 
                text-align: justify;
                text-justify: inter-word;
            }}
            h1, h2, h3 {{ 
                color: #333; 
                text-align: left;
            }}
            h1 {{ border-bottom: 2px solid #333; }}
            h2 {{ border-bottom: 1px solid #666; }}
            ul {{ 
                margin-left: 20px; 
                text-align: justify;
            }}
            li {{
                text-align: justify;
                margin-bottom: 3px;
            }}
            p {{
                text-align: justify;
                text-justify: inter-word;
            }}
            strong {{ font-weight: bold; }}
        </style>
    </head>
    <body>
        {html}
    </body>
    </html>
    """
    
    # Convert to PDF
    HTML(string=styled_html).write_pdf(output_path)


# Overview of the process

The structure is as follows:

- get information about the company (values and specialization - it uses standard web search)
- reads the resume (a word document with all the information we have)
- reads the job description as a simple text file
- generates the resume. Includes a summary paragraph
- generates a cover letter.
- passes it through an agent that acts as a recruiter
- passes it through an ATS generator to confirm that it is ATS compatible


## Get the job description details

In [10]:
#parse the page with the job description
# langchain gets the public description of the job
# make sure we are not behind a proxy
from langchain_community.document_loaders import WebBaseLoader
print(f"Getting information for job {jd_url}")
loader = WebBaseLoader(jd_url )
loader.default_parser="html.parser"
docs = loader.load()
# docs[0]


# from langchain_unstructured import UnstructuredLoader
# loader = UnstructuredLoader(web_url=jd_url)
# docs=loader.load()

description = docs[0].page_content


Getting information for job https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4314922582


In [14]:

# description="\n".join([d.page_content for d in docs[0]])
# print(f"Job Description\n{description} ")
docs

[Document(metadata={'source': 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4314922582'}, page_content='\n\n\n\n\n\n\n\n\n\nGroup Head - Scientific Innovation (RX & AI and Computational Sciences)\n\n\n\n\n\n\n\n                Novartis\n              \n\n\n              Basel, Switzerland\n            \n\n\n\n          \n\n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n\n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n\n      1 week ago\n  \n        \n\n          26 applicants\n        \n\n\n\n\n\n\n\n\n          See who Novartis has hired for this role\n        \n\n\n\n\n\n\n      Apply\n      \n\n\n\n\n \n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJoin or sign in to find your next job\nJoin to apply for the Group Head - Scientific Innovation (RX & AI and Computational Sciences) role at Novartis\n\n\n\n\n\n\n\n\n\n\n\n               

In [9]:
# ask an LLM to extrac the following information:
# job title
# job description
# job level 
# and return it as a json string
prompt=f"""
Parse the following job description text 

```
{description}
```

and extract the following information
company name.
job title.
job description and required qualifications, if the text includes description about the benefits
and company values don't include them in the description. THe job description is typically found
under sections like "Responsibilities", "What you will do", "Qualifications", "Summary", "Role description" etc.
seniority level of the job (e.g Senior, Director etc).

return the results as a json string that can be parsed directly using the keys `company_name`, `job_title`, 
`job_description` and `seniority_level` (e.g Senior, Director etc). If some information is missing return null for that field.

"""

# we use the same model as before but we dont use any tools
agent=create_agent(model=model,
                   tools=[],
                   system_prompt="You are a helpful assistant that extracts information from job descriptions. Be consice and accurate."
)
response = agent.invoke(
  {"messages": [{'role':'user', 'content': prompt}]}
)


NameError: name 'model' is not defined

In [ ]:
#parse out the JSON components
job_description=strip_markdown_backticks(
  response['messages'][-1].content
)
jd=json.loads(job_description)
# print(f"{job_description}")

{
  "company_name": "Novartis",
  "job_title": "Group Head - Scientific Innovation (RX & AI and Computational Sciences)",
  "job_description": "Novartis has embraced a bold strategy to drive a company-wide digital transformation. Our objective is to position Novartis as an industry leader by proactively adopting digital technologies that foster innovative approaches to hasten drug discovery and development. By utilizing both internal and external R&D data with the power of data science, predictive models, generative AI, and machine learning, our objective is to identify new targets, create more effective therapeutic molecules, better predict drug pharmacokinetics and safety risks, refine clinical trial design, and significantly shorten development cycles. The AICS team leads BR in exploring and applying advanced AI and ML methodologies to generate novel drug discovery insights, and to speed and improve drug discovery efficiency whilst focusing on patients’ needs.AI & Computational Scie

## Load the resume 
Let's load the document that contains the information about the resume. It is expected to be a Word document
or a PDF document.
We can use the parsers as tools in an agent, and expect the LLM to decide if it needs to parse the corresponding document. But this way we need to make a call to the LLM.
To avoid using extra API calls, we will parse the document locally, and incorporate the contents in the context of the prompt.


In [42]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import PyPDFLoader

if resume.endswith(".docx"):
  loader=Docx2txtLoader( resume )
if resume.endswith(".pdf"):
  loader=PyPDFLoader(resume)
resume_data=loader.load()



Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)


In [ ]:

resume_text=""
resume_data
for p in resume_data:
  resume_text = resume_text + p.page_content


"Konstantinos Mavrommatis, PhD Bioinformatics expert | Data Scientist  Areuse, NE, Switzerland       |     mavrommatis.konstantinos@gmail.com  +41 79 248 9368  | /in/mavrommatiskonstantinos/  |  scholar.google.com/citations  Professional summary  Highly accomplished Bioinformatician with over 20 years of experience in genomics, metagenomics, and multi-omics data integration across academic, biotech, and pharmaceutical environments. Proven expertise in developing state-of-the-art bioinformatics pipelines, establishing robust data management systems, and deploying scalable HPC/cloud infrastructures to accelerate discovery and innovation. Extensive track record in multi-omics integration (genomics, transcriptomics, proteomics, metabolomics) for biomarker discovery, microbiome research, and translational science. Recognized for 350+ scientiﬁc publications, leadership of cross-functional teams, and delivery of innovative bioinformatics solutions that enable reproducibility, scalability, and

## Search for the company of interest

Get informaation about the company that is hiring. 
We are specifically looking for their values and mission statement, this can be used to tailor the resume and the cover letter.

In [45]:
# create the search with the necessary prompt 
# to get information about the values of the company
search_tool=TavilySearchResults(max_results=2)

# test how it works:
# search_results=search.invoke( f"What is the focus of the company {company} and what are its values.")
# print(f"Search returned\n{search_results}")

/var/folders/h7/b5y3fd8s671_5jhxlq2cyj7c0000gn/T/ipykernel_82505/3200585184.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool=TavilySearchResults(max_results=2)


In [52]:
tools=[search_tool]
prompt=f"""What is company {jd.get('company_name')} specialized on?
 What are the values they are proud of?
 
 Return the results as a json string with keys:
 company_name,
 specialized_in which will be a string describing what the company specializes in
 values: a list of strings describing the values of the company
 """
agent=create_agent(model=model, 
                   tools=[search_tool],
                   system_prompt="You are a helpful assistant that extracts information from the internet.  Be consice and accurate."
                   )

input_message={'role':'user', 
               'content': prompt}
response = agent.invoke({"messages": [input_message]})

In [83]:
company_values=strip_markdown_backticks(
  response['messages'][-1].content
)
company_values=json.loads(company_values)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# print(f"For the company {jd.get('company_name')} the values we got are {company_values}")

For the company Novartis the values we got are {
 "company_name": "Novartis",
 "specialized_in": "Novartis is a global healthcare company that provides medicines and treatments for various diseases and health conditions. They focus on research and development to create innovative therapies.",
 "values": [
  "Innovation",
  "Quality",
  "Collaboration",
  "Performance",
  "Courage",
  "Integrity"
 ]
}


## Create a professional summary based on the experience

Using the working experience from the resume, create a professional summary that can be used at the top of the resume.

In [ ]:
prompt=f"""
Read the working experience and education from my resume:
```
{resume_text}
```
and create a compelling professional summary for a job application 
that fits the job description below as a reference but don't mention the job title 
or the company name.
and keep it to 500 characters or less. 
The job description is:
```
{jd.get('job_description')}
```
"""

agent=create_agent(model, 
                   tools=[],
                   system_prompt="You are an expert career coach. Be concise and critical. Don't be sycophantic."
) 

response = agent.invoke(
  {"messages": [{'role':'user', 'content': prompt}]}
)

In [ ]:
professional_summary=response['messages'][-1].content

# print(f"Professional Summary:\n{professional_summary}")

Professional Summary:
Bioinformatics leader with 20+ years in multi-omics and AI/ML for drug discovery. Proven ability to design and deploy scalable data platforms, accelerate biomarker discovery, and lead cross-functional teams. Expertise in integrating genomics, transcriptomics, and AI to drive innovation and shorten development cycles, aligning with Novartis' digital transformation goals.


## Work out the resume 

Create the resume based on the job description, the company values and specialization, and the professional summary.

In [60]:
prompt=f"""
Write a tailored resume for this {jd.get("job_title")} role at 
{jd.get("company_name")}. 
Don't include an objective statement or references, 
but do include the professional summary {professional_summary}, 
Use my past work experience to extract 3-5 bullet points per role, 
and incorporate the most important keywords from the job description 
in those achievements. 
The job description is: {jd.get('job_description')}
My resume is as follows: {resume_text}
The company values are: {company_values}
Focus on aligning my skills and experiences with the job requirements.
Categorize the skills section into Technical Skills and Soft Skills.

Return the resume in markdown format.
include the sections
- Professional Summary
- Work Experience
- Education
- Skills
- Awards
- Languages
- Interests
"""

agent=create_agent(model=model, 
                   tools=[],
                   system_prompt="You are an expert career coach.  Be consice and accurate. Don't be sycophantic. "
                   )

input_message={'role':'user', 
               'content': prompt}
response = agent.invoke({"messages": [input_message]})

In [ ]:
raw_resume=response['messages'][-1].content

# print(f"Raw resume:\n{raw_resume}")

Raw resume:
```markdown
# Konstantinos Mavrommatis, PhD

**Bioinformatics Leader | Data Scientist**

Areuse, NE, Switzerland | mavrommatis.konstantinos@gmail.com | +41 79 248 9368 | /in/mavrommatiskonstantinos/ | scholar.google.com/citations

---

## Professional Summary

Bioinformatics leader with 20+ years in multi-omics and AI/ML for drug discovery. Proven ability to design and deploy scalable data platforms, accelerate biomarker discovery, and lead cross-functional teams. Expertise in integrating genomics, transcriptomics, and AI to drive innovation and shorten development cycles, aligning with Novartis' digital transformation goals.

---

## Work Experience

**Scientific Director, Applied Bioinformatics** | Bristol Myers Squibb (formerly Celgene) | Boudry, CH / San Francisco, CA, USA | 2020 – Present

*   Directed strategic initiatives as a primary liaison between SMEs, IT, and R&D, designing and deploying cutting-edge data science and AI capabilities to accelerate drug discovery 

## Fine tune the resume 

Change the resume language to be more to the point


In [63]:
prompt=f"""
 Can you audit this entire resume {raw_resume} and point out areas where 
 I am being too vague, too wordy, or not showing enough impact? 

 I want your feedback on tone, structure, and how I can better emphasize 
 leadership, results, or innovation.
 Please act like a hiring manager in industry for a {jd.get('job_level')} position. 
 Based on this resume, what would make you more likely to invite me 
 for an interview? What should I change, cut, or add to improve my chances?
"""

agent=create_agent(model=model, 
                   tools=[],
                   system_prompt="You are an expert recruiter in a company. Be consice and accurate. Don't be sycophantic."
                   )

input_message={'role':'user', 
               'content': prompt}
response = agent.invoke({"messages": [input_message]})

In [64]:
fine_tune_points=response['messages'][-1].content

print(f"Fine tune points:\n{fine_tune_points}")

Fine tune points:
This is a strong resume showcasing significant experience. Here's an audit focusing on vagueness, wordiness, impact, and areas for improvement, with a focus on emphasizing leadership, results, and innovation.

## Resume Audit: Konstantinos Mavrommatis, PhD

**Overall Impression:** The resume presents a highly experienced and accomplished bioinformatics leader with a clear trajectory in drug discovery and data science. The structure is logical, and the content demonstrates a deep understanding of the field.

**Tone:** Professional and results-oriented.

**Structure:** Well-organized and easy to follow.

---

### Areas for Improvement:

**1. Professional Summary:**

*   **Vagueness/Wordiness:** "Proven ability to design and deploy scalable data platforms, accelerate biomarker discovery, and lead cross-functional teams." While true, it's a bit generic.
*   **Impact:** Could be more specific about *what kind* of drug discovery and *what scale* of impact.
*   **Leadership/

In [65]:
prompt=f"""
Fine tune the following  resume for this {jd.get("job_title")} role at 
{jd.get("company_name")}. 

The job description is: {jd.get('job_description')}
My resume is as follows: {raw_resume}
The company values are: {company_values}
Focus on aligning my skills and experiences with the job requirements
and take into account the following feedback from a hiring manager:
```
{fine_tune_points}
```
Categorize the skills section into Technical Skills and Soft Skills.

Return the resume in markdown format.
include the sections
- Professional Summary
- Work Experience
- Education
- Skills
- Awards
- Languages
- Interests
"""
agent=create_agent(model=model, 
                   tools=[],
                   system_prompt="You are an expert career coach.  Be consice and accurate. Don't be sycophantic. "
                   )
input_message={'role':'user',
                'content': prompt}
response = agent.invoke({"messages": [input_message]})


In [ ]:
raw_resume=response['messages'][-1].content


## Work on the formatting of the resume

Make sure it is in the ATS combatible format


In [66]:

prompt=f"""

proofread my resume 
```
{raw_resume} 
```
and suggest improvements for clarity and readability.
Check my resume for passive voice and rewrite it using stronger action verbs. 
Review my resume for redundant phrasing and suggest more concise alternatives.
Check my resume for consistency in formatting, punctuation, and capitalization.
Identify areas where I can add quantifiable metrics to strengthen my resume.
Please check the spelling and grammar of my resume to ensure it's correct. 
If necessary rewrite the resume bullet points in a more engaging 
and natural way. Use nondramatic language.
Vary sentence lengths and structures instead of following predictable patterns
Make my resume sound less generic while still maintaining professionalism.
Adjust my resume tone to sound more natural and engaging, 
avoiding stiff or overly polished language.
Edit my resume to include a mix of sentence structures that 
feel more natural and human-written.

Do not change the overall structure of the resume and its sections.

Return the resume in markdown format.
"""


agent=create_agent(model=model, 
                   tools=[],
                   system_prompt="You are an expert career coach and linguistics professional.  Be consice and accurate. Don't be sycophantic. "
                   )
input_message={'role':'user',
                'content': prompt}
response = agent.invoke({"messages": [input_message]})

In [67]:
fine_tuned_resume=response['messages'][-1].content

## Make sure the resume is ATS compatible

In [68]:
pass_ats=f"""
Optimize my resume to pass Applicant Tracking Systems (ATS) 
for the {jd.get('job_title')} role at 
{jd.get('company_name')}. 
Incorporate relevant keywords and phrases from the job description: {jd.get('job_description')}.
Ensure the formatting is ATS-friendly by avoiding complex layouts, graphics, and tables.
My current resume is: {fine_tuned_resume}

Return the resume in markdown format.
"""

agent=create_agent(model=model, 
                   tools=[],
                   system_prompt="You are file format expert. Be consice and accurate. Don't be sycophantic. Double check every suggestion. "
                   )
input_message={'role':'user',
                'content': pass_ats}
response = agent.invoke({"messages": [input_message]})

In [75]:
final_resume=response['messages'][-1].content

final_resume=strip_markdown_backticks(final_resume)

output_pdf=f"Resume_{jd.get('company_name').replace(' ','_')}.pdf"
markdown_to_pdf(final_resume, output_pdf)
print(f"Final resume saved to {output_pdf}")


Final resume saved to Resume_Novartis.pdf


## Prepare a cover letter

Write a cover letter based on the resume and the job description.

In [78]:
prompt=f"""
Draft a cover letter based on the resume and the job description.
The job description is:
```
{jd.get('job_description')}
``` 
and my resume is
```
{final_resume} 
```.

Focus on aligning my skills and experiences with the job requirements 
and the company values {company_values.get('company_values')}.
Emphasize how my background makes me a strong fit for the role at
{jd.get('company_name')}.

Structure the cover letter with an introduction, body paragraphs highlighting key qualifications,
and a conclusion expressing enthusiasm for the position.
Avoid generic statements and tailor the content specifically to the job and company.

Return the cover letter in markdown format.
"""



agent=create_agent(model=model, 
                   tools=[],
                   system_prompt="You are an expert hiring manager. Be consice and accurate. Don't be sycophantic."
                   )

input_message={'role':'user',
                'content': prompt}  
response = agent.invoke({"messages": [input_message]})


AttributeError: 'str' object has no attribute 'get'

In [ ]:
cover_letter=response['messages'][-1].content

cover_letter=strip_markdown_backticks(cover_letter)
output_pdf=f"Cover_Letter_{jd.get('company_name').replace(' ','_')}.pdf"
markdown_to_pdf(cover_letter, output_pdf)
print(f"Final cover letter saved to {output_pdf}")